# Summarization

## Deep Reinforced Model for Abstractive Summarization

[A Deep Reinforced Model for Abstractive Summarization (Romain Paulus, Caiming Xiong, Richard Socher, 2017)](https://arxiv.org/abs/1705.04304) - модель суммаризации на основе encoder-decoder с использованием reinforcement learning для обучения.

### Источники
1. Блог с описанием статьи: https://blog.einstein.ai/your-tldr-by-an-ai-a-deep-reinforced-model-for-abstractive-summarization/
2. Имплементация: https://github.com/rohithreddy024/Text-Summarizer-Pytorch

In [1]:
!git clone https://github.com/rohithreddy024/Text-Summarizer-Pytorch

Cloning into 'Text-Summarizer-Pytorch'...
remote: Enumerating objects: 98, done.
remote: Total 98 (delta 0), reused 0 (delta 0), pack-reused 98
Unpacking objects: 100% (98/98), done.


### Данные 
Модель обучается на данных Gigaword dataset: https://github.com/harvardnlp/sent-summary

In [2]:
# find the share link of the file/folder on Google Drive
file_share_link = "https://drive.google.com/open?id=0B6N7tANPyVeBNmlSX19Ld2xDU1E"

# extract the ID of the file
file_id = file_share_link[file_share_link.find("=") + 1:]

# append the id to this REST command
file_download_link = "https://docs.google.com/uc?export=download&id=" + file_id 

In [3]:
file_id

'0B6N7tANPyVeBNmlSX19Ld2xDU1E'

In [4]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=0B6N7tANPyVeBNmlSX19Ld2xDU1E' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=0B6N7tANPyVeBNmlSX19Ld2xDU1E" -O summary.tar.gz && rm -rf /tmp/cookies.txt

--2020-11-07 16:24:21--  https://docs.google.com/uc?export=download&confirm=kkak&id=0B6N7tANPyVeBNmlSX19Ld2xDU1E
Resolving docs.google.com (docs.google.com)... 172.217.5.238, 2607:f8b0:4004:809::200e
Connecting to docs.google.com (docs.google.com)|172.217.5.238|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-0s-docs.googleusercontent.com/docs/securesc/vap1rridkhuhd0re3ocuc8hn139v6rp0/sgd4hnuddek2mr6a1sktuavgmfnrp02g/1604766225000/03129501499031348422/09442447318250874164Z/0B6N7tANPyVeBNmlSX19Ld2xDU1E?e=download [following]
--2020-11-07 16:24:21--  https://doc-10-0s-docs.googleusercontent.com/docs/securesc/vap1rridkhuhd0re3ocuc8hn139v6rp0/sgd4hnuddek2mr6a1sktuavgmfnrp02g/1604766225000/03129501499031348422/09442447318250874164Z/0B6N7tANPyVeBNmlSX19Ld2xDU1E?e=download
Resolving doc-10-0s-docs.googleusercontent.com (doc-10-0s-docs.googleusercontent.com)... 172.217.15.97, 2607:f8b0:4004:811::2001
Connecting to doc-10-0s-docs.googleus

In [5]:
!tar -tvf summary.tar.gz

drwxrwxr-x srush/srush       0 2016-04-12 15:46 sumdata/
drwxrwxr-x srush/srush       0 2016-04-12 15:27 sumdata/DUC2004/
-rw-r--r-- srush/srush  103953 2016-04-12 15:27 sumdata/DUC2004/input.txt
-rw-r--r-- srush/srush   36083 2016-04-12 15:27 sumdata/DUC2004/task1_ref3.txt
-rw-r--r-- srush/srush   34426 2016-04-12 15:27 sumdata/DUC2004/task1_ref0.txt
-rw-r--r-- srush/srush   35530 2016-04-12 15:27 sumdata/DUC2004/task1_ref1.txt
-rw-r--r-- srush/srush   35967 2016-04-12 15:27 sumdata/DUC2004/task1_ref2.txt
drwxrwxr-x srush/srush       0 2016-04-12 15:30 sumdata/Giga/
-rw-rw-r-- srush/srush  335302 2016-04-12 15:25 sumdata/Giga/input.txt
-rw-rw-r-- srush/srush  103766 2016-04-12 15:28 sumdata/Giga/task1_ref0.txt
drwxrwxr-x srush/srush       0 2016-04-12 15:46 sumdata/DUC2003/
-rw-r--r-- srush/srush  128364 2016-04-12 15:46 sumdata/DUC2003/input.txt
-rw-r--r-- srush/srush   48910 2016-04-12 15:46 sumdata/DUC2003/task1_ref3.txt
-rw-r--r-- srush/srush   43027 2016-04-12 15:46 sumdata/DUC20

In [6]:
!tar -xf summary.tar.gz sumdata/train/
!gunzip sumdata/train/train.article.txt.gz
!gunzip sumdata/train/train.title.txt.gz
! mv sumdata/train/* Text-Summarizer-Pytorch/data/unfinished
!rm -rf sumdata/

In [7]:
%cd Text-Summarizer-Pytorch

/content/Text-Summarizer-Pytorch


In [8]:
# Создание .bin файлов с данными для обучения модели
!python2 make_data_files.py

Completed shuffling train & valid text files
3803957it [02:43, 23208.87it/s]
189651it [00:03, 60835.47it/s]
Completed creating bin file for train & valid
Completed chunking main bin files into smaller ones


Примеры обучающих данных (заголовки и абстракты статей):

In [9]:
N = 20
with open("data/unfinished/train.article.txt") as f:
    head = [next(f) for x in range(N)]

In [10]:
print(head[1])

at least two people were killed in a suspected bomb attack on a passenger bus in the strife-torn southern philippines on monday , the military said .



In [11]:
N = 20
with open("data/unfinished/train.title.txt") as f:
    head = [next(f) for x in range(N)]
print(head[1])

at least two dead in southern philippines blast



### Модель
Идея состоит в обучении encoder-decoder архитектуры для генерации summary входного текста. 

В декодере дважды используется механизм attention:
1. attention на состояния энкодера (intra-temporal attention) определяет вес слов входной последовательности для текущей позиции в выходной последовательности summary
2. attention на предыдущие состояния декодера (intra-decoder attention) для того, чтобы не допускать повторения слов в выходе декодера.

В процессе обучения модели используется teacher-forcing, чтобы учитывать ошибку на уровне каждого генерируемого слова (Negative Log Likelihood Loss), и reinforcement learning для оценки качества сгенерированного текста целиком в сравнении с target summary. 

Для reinforcement learning в качестве метрики используется ROUGE score. ROUGE считает совпадение н-грамм слов в таргете и сгенерированной последовательности (ROUGE-1 для униграмм, ROUGE-2 для биграмм слов, ...). 


![summ_attentions](summ-attentions.svg)

In [12]:
# rouge для подчета метрики Rouge
!pip install rouge

Обучение модели (train.py)

In [14]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"    #Set cuda device

import time

import torch as T
import torch.nn as nn
import torch.nn.functional as F
from model import Model

from data_util import config, data
from data_util.batcher import Batcher
from data_util.data import Vocab
from train_util import *
from torch.distributions import Categorical
from rouge import Rouge
from numpy import random
import argparse

random.seed(123)
T.manual_seed(123)
if T.cuda.is_available():
    T.cuda.manual_seed_all(123)

class Train(object):
    def __init__(self, opt):
        self.vocab = Vocab(config.vocab_path, config.vocab_size)
        self.batcher = Batcher(config.train_data_path, self.vocab, mode='train',
                               batch_size=config.batch_size, single_pass=False)
        self.opt = opt
        self.start_id = self.vocab.word2id(data.START_DECODING)
        self.end_id = self.vocab.word2id(data.STOP_DECODING)
        self.pad_id = self.vocab.word2id(data.PAD_TOKEN)
        self.unk_id = self.vocab.word2id(data.UNKNOWN_TOKEN)
        time.sleep(5)

    def save_model(self, iter):
        save_path = config.save_model_path + "/%07d.tar" % iter
        T.save({
            "iter": iter + 1,
            "model_dict": self.model.state_dict(),
            "trainer_dict": self.trainer.state_dict()
        }, save_path)

    def setup_train(self):
        self.model = Model()
        self.model = get_cuda(self.model)
        self.trainer = T.optim.Adam(self.model.parameters(), lr=config.lr)
        start_iter = 0
        if self.opt.load_model is not None:
            load_model_path = os.path.join(config.save_model_path, self.opt.load_model)
            checkpoint = T.load(load_model_path)
            start_iter = checkpoint["iter"]
            self.model.load_state_dict(checkpoint["model_dict"])
            self.trainer.load_state_dict(checkpoint["trainer_dict"])
            print("Loaded model at " + load_model_path)
        if self.opt.new_lr is not None:
            self.trainer = T.optim.Adam(self.model.parameters(), lr=self.opt.new_lr)
        return start_iter

    def train_batch_MLE(self, enc_out, enc_hidden, enc_padding_mask, ct_e, extra_zeros, enc_batch_extend_vocab, batch):
        ''' Calculate Negative Log Likelihood Loss for the given batch. In order to reduce exposure bias,
                pass the previous generated token as input with a probability of 0.25 instead of ground truth label
        Args:
        :param enc_out: Outputs of the encoder for all time steps (batch_size, length_input_sequence, 2*hidden_size)
        :param enc_hidden: Tuple containing final hidden state & cell state of encoder. Shape of h & c: (batch_size, hidden_size)
        :param enc_padding_mask: Mask for encoder input; Tensor of size (batch_size, length_input_sequence) with values of 0 for pad tokens & 1 for others
        :param ct_e: encoder context vector for time_step=0 (eq 5 in https://arxiv.org/pdf/1705.04304.pdf)
        :param extra_zeros: Tensor used to extend vocab distribution for pointer mechanism
        :param enc_batch_extend_vocab: Input batch that stores OOV ids
        :param batch: batch object
        '''
        dec_batch, max_dec_len, dec_lens, target_batch = get_dec_data(batch)                        #Get input and target batchs for training decoder
        step_losses = []
        s_t = (enc_hidden[0], enc_hidden[1])                                                        #Decoder hidden states
        x_t = get_cuda(T.LongTensor(len(enc_out)).fill_(self.start_id))                             #Input to the decoder
        prev_s = None                                                                               #Used for intra-decoder attention (section 2.2 in https://arxiv.org/pdf/1705.04304.pdf)
        sum_temporal_srcs = None                                                                    #Used for intra-temporal attention (section 2.1 in https://arxiv.org/pdf/1705.04304.pdf)
        for t in range(min(max_dec_len, config.max_dec_steps)):
            use_gound_truth = get_cuda((T.rand(len(enc_out)) > 0.25)).long()                        #Probabilities indicating whether to use ground truth labels instead of previous decoded tokens
            x_t = use_gound_truth * dec_batch[:, t] + (1 - use_gound_truth) * x_t                   #Select decoder input based on use_ground_truth probabilities
            x_t = self.model.embeds(x_t)
            final_dist, s_t, ct_e, sum_temporal_srcs, prev_s = self.model.decoder(x_t, s_t, enc_out, enc_padding_mask, ct_e, extra_zeros, enc_batch_extend_vocab, sum_temporal_srcs, prev_s)
            target = target_batch[:, t]
            log_probs = T.log(final_dist + config.eps)
            step_loss = F.nll_loss(log_probs, target, reduction="none", ignore_index=self.pad_id)
            step_losses.append(step_loss)
            x_t = T.multinomial(final_dist, 1).squeeze()                                            #Sample words from final distribution which can be used as input in next time step
            is_oov = (x_t >= config.vocab_size).long()                                              #Mask indicating whether sampled word is OOV
            x_t = (1 - is_oov) * x_t.detach() + (is_oov) * self.unk_id                              #Replace OOVs with [UNK] token

        losses = T.sum(T.stack(step_losses, 1), 1)                                                  #unnormalized losses for each example in the batch; (batch_size)
        batch_avg_loss = losses / dec_lens                                                          #Normalized losses; (batch_size)
        mle_loss = T.mean(batch_avg_loss)                                                           #Average batch loss
        return mle_loss

    def train_batch_RL(self, enc_out, enc_hidden, enc_padding_mask, ct_e, extra_zeros, enc_batch_extend_vocab, article_oovs, greedy):
        '''Generate sentences from decoder entirely using sampled tokens as input. These sentences are used for ROUGE evaluation
        Args
        :param enc_out: Outputs of the encoder for all time steps (batch_size, length_input_sequence, 2*hidden_size)
        :param enc_hidden: Tuple containing final hidden state & cell state of encoder. Shape of h & c: (batch_size, hidden_size)
        :param enc_padding_mask: Mask for encoder input; Tensor of size (batch_size, length_input_sequence) with values of 0 for pad tokens & 1 for others
        :param ct_e: encoder context vector for time_step=0 (eq 5 in https://arxiv.org/pdf/1705.04304.pdf)
        :param extra_zeros: Tensor used to extend vocab distribution for pointer mechanism
        :param enc_batch_extend_vocab: Input batch that stores OOV ids
        :param article_oovs: Batch containing list of OOVs in each example
        :param greedy: If true, performs greedy based sampling, else performs multinomial sampling
        Returns:
        :decoded_strs: List of decoded sentences
        :log_probs: Log probabilities of sampled words
        '''
        s_t = enc_hidden                                                                            #Decoder hidden states
        x_t = get_cuda(T.LongTensor(len(enc_out)).fill_(self.start_id))                             #Input to the decoder
        prev_s = None                                                                               #Used for intra-decoder attention (section 2.2 in https://arxiv.org/pdf/1705.04304.pdf)
        sum_temporal_srcs = None                                                                    #Used for intra-temporal attention (section 2.1 in https://arxiv.org/pdf/1705.04304.pdf)
        inds = []                                                                                   #Stores sampled indices for each time step
        decoder_padding_mask = []                                                                   #Stores padding masks of generated samples
        log_probs = []                                                                              #Stores log probabilites of generated samples
        mask = get_cuda(T.LongTensor(len(enc_out)).fill_(1))                                        #Values that indicate whether [STOP] token has already been encountered; 1 => Not encountered, 0 otherwise

        for t in range(config.max_dec_steps):
            x_t = self.model.embeds(x_t)
            probs, s_t, ct_e, sum_temporal_srcs, prev_s = self.model.decoder(x_t, s_t, enc_out, enc_padding_mask, ct_e, extra_zeros, enc_batch_extend_vocab, sum_temporal_srcs, prev_s)
            if greedy is False:
                multi_dist = Categorical(probs)
                x_t = multi_dist.sample()                                                           #perform multinomial sampling
                log_prob = multi_dist.log_prob(x_t)
                log_probs.append(log_prob)
            else:
                _, x_t = T.max(probs, dim=1)                                                        #perform greedy sampling
            x_t = x_t.detach()
            inds.append(x_t)
            mask_t = get_cuda(T.zeros(len(enc_out)))                                                #Padding mask of batch for current time step
            mask_t[mask == 1] = 1                                                                   #If [STOP] is not encountered till previous time step, mask_t = 1 else mask_t = 0
            mask[(mask == 1) + (x_t == self.end_id) == 2] = 0                                       #If [STOP] is not encountered till previous time step and current word is [STOP], make mask = 0
            decoder_padding_mask.append(mask_t)
            is_oov = (x_t>=config.vocab_size).long()                                                #Mask indicating whether sampled word is OOV
            x_t = (1-is_oov)*x_t + (is_oov)*self.unk_id                                             #Replace OOVs with [UNK] token

        inds = T.stack(inds, dim=1)
        decoder_padding_mask = T.stack(decoder_padding_mask, dim=1)
        if greedy is False:                                                                         #If multinomial based sampling, compute log probabilites of sampled words
            log_probs = T.stack(log_probs, dim=1)
            log_probs = log_probs * decoder_padding_mask                                            #Not considering sampled words with padding mask = 0
            lens = T.sum(decoder_padding_mask, dim=1)                                               #Length of sampled sentence
            log_probs = T.sum(log_probs, dim=1) / lens  # (bs,)                                     #compute normalizied log probability of a sentence
        decoded_strs = []
        for i in range(len(enc_out)):
            id_list = inds[i].cpu().numpy()
            oovs = article_oovs[i]
            S = data.outputids2words(id_list, self.vocab, oovs)                                     #Generate sentence corresponding to sampled words
            try:
                end_idx = S.index(data.STOP_DECODING)
                S = S[:end_idx]
            except ValueError:
                S = S
            if len(S) < 2:                                                                           #If length of sentence is less than 2 words, replace it with "xxx"; Avoids setences like "." which throws error while calculating ROUGE
                S = ["xxx"]
            S = " ".join(S)
            decoded_strs.append(S)

        return decoded_strs, log_probs

    def reward_function(self, decoded_sents, original_sents):
        rouge = Rouge()
        try:
            scores = rouge.get_scores(decoded_sents, original_sents)
        except Exception:
            print("Rouge failed for multi sentence evaluation.. Finding exact pair")
            scores = []
            for i in range(len(decoded_sents)):
                try:
                    score = rouge.get_scores(decoded_sents[i], original_sents[i])
                except Exception:
                    print("Error occured at:")
                    print("decoded_sents:", decoded_sents[i])
                    print("original_sents:", original_sents[i])
                    score = [{"rouge-l":{"f":0.0}}]
                scores.append(score[0])
        rouge_l_f1 = [score["rouge-l"]["f"] for score in scores]
        rouge_l_f1 = get_cuda(T.FloatTensor(rouge_l_f1))
        return rouge_l_f1

    # def write_to_file(self, decoded, max, original, sample_r, baseline_r, iter):
    #     with open("temp.txt", "w") as f:
    #         f.write("iter:"+str(iter)+"\n")
    #         for i in range(len(original)):
    #             f.write("dec: "+decoded[i]+"\n")
    #             f.write("max: "+max[i]+"\n")
    #             f.write("org: "+original[i]+"\n")
    #             f.write("Sample_R: %.4f, Baseline_R: %.4f\n\n"%(sample_r[i].item(), baseline_r[i].item()))


    def train_one_batch(self, batch, iter):
        enc_batch, enc_lens, enc_padding_mask, enc_batch_extend_vocab, extra_zeros, context = get_enc_data(batch)

        enc_batch = self.model.embeds(enc_batch)                                                    #Get embeddings for encoder input
        enc_out, enc_hidden = self.model.encoder(enc_batch, enc_lens)

        # -------------------------------Summarization-----------------------
        if self.opt.train_mle == "yes":                                                             #perform MLE training
            mle_loss = self.train_batch_MLE(enc_out, enc_hidden, enc_padding_mask, context, extra_zeros, enc_batch_extend_vocab, batch)
        else:
            mle_loss = get_cuda(T.FloatTensor([0]))
        # --------------RL training-----------------------------------------------------
        if self.opt.train_rl == "yes":                                                              #perform reinforcement learning training
            # multinomial sampling
            sample_sents, RL_log_probs = self.train_batch_RL(enc_out, enc_hidden, enc_padding_mask, context, extra_zeros, enc_batch_extend_vocab, batch.art_oovs, greedy=False)
            with T.autograd.no_grad():
                # greedy sampling
                greedy_sents, _ = self.train_batch_RL(enc_out, enc_hidden, enc_padding_mask, context, extra_zeros, enc_batch_extend_vocab, batch.art_oovs, greedy=True)

            sample_reward = self.reward_function(sample_sents, batch.original_abstracts)
            baseline_reward = self.reward_function(greedy_sents, batch.original_abstracts)
            # if iter%200 == 0:
            #     self.write_to_file(sample_sents, greedy_sents, batch.original_abstracts, sample_reward, baseline_reward, iter)
            rl_loss = -(sample_reward - baseline_reward) * RL_log_probs                             #Self-critic policy gradient training (eq 15 in https://arxiv.org/pdf/1705.04304.pdf)
            rl_loss = T.mean(rl_loss)

            batch_reward = T.mean(sample_reward).item()
        else:
            rl_loss = get_cuda(T.FloatTensor([0]))
            batch_reward = 0

    # ------------------------------------------------------------------------------------
        self.trainer.zero_grad()
        (self.opt.mle_weight * mle_loss + self.opt.rl_weight * rl_loss).backward()
        self.trainer.step()

        return mle_loss.item(), batch_reward

    def trainIters(self):
        iter = self.setup_train()
        count = mle_total = r_total = 0
        while iter <= config.max_iterations:
            batch = self.batcher.next_batch()
            try:
                mle_loss, r = self.train_one_batch(batch, iter)
            except KeyboardInterrupt:
                print("-------------------Keyboard Interrupt------------------")
                exit(0)

            mle_total += mle_loss
            r_total += r
            count += 1
            iter += 1

            if iter % 1000 == 0:
                mle_avg = mle_total / count
                r_avg = r_total / count
                print("iter:", iter, "mle_loss:", "%.3f" % mle_avg, "reward:", "%.4f" % r_avg)
                count = mle_total = r_total = 0

            if iter % 5000 == 0:
                self.save_model(iter)


# parser = argparse.ArgumentParser()
# parser.add_argument('--train_mle', type=str, default="yes")
# parser.add_argument('--train_rl', type=str, default="no")
# parser.add_argument('--mle_weight', type=float, default=1.0)
# parser.add_argument('--load_model', type=str, default=None)
# parser.add_argument('--new_lr', type=float, default=None)
# opt = parser.parse_args()
# opt.rl_weight = 1 - opt.mle_weight
# print("Training mle: %s, Training rl: %s, mle weight: %.2f, rl weight: %.2f"%(opt.train_mle, opt.train_rl, opt.mle_weight, opt.rl_weight))
# print("intra_encoder:", config.intra_encoder, "intra_decoder:", config.intra_decoder)

# train_processor = Train(opt)
# train_processor.trainIters()

Сначала encoder-decoder модель обучается без reinforcement learning. 

In [15]:
!python train.py --train_mle=yes --train_rl=no --mle_weight=1.0

2020-11-07 16:57:01.568167: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Training mle: yes, Training rl: no, mle weight: 1.00, rl weight: 0.00
intra_encoder: True intra_decoder: True
Exception in thread Thread-3:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/content/Text-Summarizer-Pytorch/data_util/batcher.py", line 247, in watch_threads
    tf.logging.info(
AttributeError: module 'tensorflow' has no attribute 'logging'

-------------------Keyboard Interrupt------------------


Выбирается лучшая модель (из обученных с разным числом итераций) по значению ROUGE на валидационной выборке.

In [ ]:
!python eval.py --task=validate --start_from=0005000.tar

Лучшая модель дообучается с использованием RL.

In [ ]:
# MLE + RL training
!python train.py --train_mle=yes --train_rl=yes --mle_weight=0.25 --load_model=0100000.tar --new_lr=0.0001 

# RL training
python train.py --train_mle=no --train_rl=yes --mle_weight=0.0 --load_model=0100000.tar --new_lr=0.0001

Модель, обученная только на RL, достигает более высоких показателей ROUGE, но генерирует менее хорошие тексты с точки зрения связности и естественности, поэтому авторы статьи рекомендуют комбинированную стратегию обучения. 

* Результаты, приведенные авторами репозитория:

Rouge scores obtained by using best MLE trained model on test set:

{
'rouge-1': {'f': 0.4412018559893622, 'p': 0.4814799494024485, 'r': 0.4232331027817015}, 

'rouge-2': {'f': 0.23238981595683728, 'p': 0.2531296070596062, 'r': 0.22407861554997008},

'rouge-l': {'f': 0.40477682528278364, 'p': 0.4584684491434479, 'r': 0.40351107200202596}
}


Rouge scores obtained by using best MLE + RL trained model on test set:

{
'rouge-1': {'f': 0.4499047033247696, 'p': 0.4853756369556345, 'r': 0.43544461386607497},

'rouge-2': {'f': 0.24037014314625643, 'p': 0.25903387205387235, 'r': 0.23362662645146298},

'rouge-l': {'f': 0.41320241732946406, 'p': 0.4616655167980162, 'r': 0.4144419466382236}
}

* Примеры (article - исходный текст, ref - target summary, dec - сгенерированный моделью текст):

article: russia 's lower house of parliament was scheduled friday to debate an appeal to the prime minister that challenged the right of u.s.-funded radio liberty to operate in russia following its introduction of broadcasts targeting chechnya .

ref: russia 's lower house of parliament mulls challenge to radio liberty

dec: russian parliament to debate on banning radio liberty


article: continued dialogue with the democratic people 's republic of korea is important although australia 's plan to open its embassy in pyongyang has been shelved because of the crisis over the dprk 's nuclear weapons program , australian foreign minister alexander downer said on friday .

ref: dialogue with dprk important says australian foreign minister

dec: australian fm says dialogue with dprk important

article: water levels in the zambezi river are rising due to heavy rains in its catchment area , prompting zimbabwe 's civil protection unit -lrb- cpu -rrb- to issue a flood alert for people living in the zambezi valley , the herald reported on friday .

ref: floods loom in zambezi valley

dec: water levels rising in zambezi river

## BERT Extractive Summarization

### Источник:
https://deeplearninganalytics.org/text-summarization/

https://github.com/nlpyang/BertSum


Идея: использовать BERT эмбеддинги предложений исходного текста в задаче бинарной классификации для отбора самых значимых предложений, которые войдут в summary.

Для получения эмбеддингов нескольких предложений текста перед каждым предложением текста вставляется свой токен начала предложения **[CLS]**, после каждого предложения - символ **[SEP]**. В качестве эмбеддингов сегмента предложения (которые используются для того, чтобы различать первое и второе предложения в парах предложений при обучении  BERT) для последовательности предложений чередуются единичные и нулевые вектора.

_[sent1, sent2, sent3, sent4, sent5] -> [EA, EB, EA, EB, EA]._

Вектора токенов [CLS] на последнем слое BERT используются в качестве векторов предложений текста. Вектора предложений подаются на вход классификатору (в статье 3 варианта классификации): 
1. linear layer + sigmoid
2. Transformer + sigmoid
3. LSTM + sigmoid

![bertsum](bertsum.png)

In [16]:
!pip install --force-reinstall torch==1.1.0

     |████████████████████████████████| 676.9MB 26kB/s 
     |████████████████████████████████| 14.5MB 223kB/s 
ERROR: torchvision 0.8.1+cu101 has requirement torch==1.7.0, but you'll have torch 1.1.0 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101


In [17]:
!pip install pytorch-pretrained-bert

     |████████████████████████████████| 133kB 11.0MB/s 
     |████████████████████████████████| 102kB 9.8MB/s 
     |████████████████████████████████| 6.7MB 22.7MB/s 
     |████████████████████████████████| 71kB 10.3MB/s 
  Created wheel for boto3: filename=boto3-1.16.13-py2.py3-none-any.whl size=128453 sha256=bdf121fdc77e33dfcfdb80dffb082cbf784eedf94d1ab692bda0ccff21687c06
  Stored in directory: /root/.cache/pip/wheels/bd/ca/cc/d99cff66806b87034af25f8fd8b0adb3e0151b17eea7891143
Successfully built boto3
ERROR: botocore 1.19.13 has requirement urllib3<1.26,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [18]:
!pip install tensorboardX

     |████████████████████████████████| 317kB 9.1MB/s 


In [19]:
!pip install pyrouge


     |████████████████████████████████| 61kB 5.9MB/s 
  Created wheel for pyrouge: filename=pyrouge-0.1.3-cp36-none-any.whl size=191613 sha256=22a94db50597fededb20b1f7822fc6a4e9531ca33696289ee453bbbe6b0fa570
  Stored in directory: /root/.cache/pip/wheels/75/d3/0c/e5b04e15b6b87c42e980de3931d2686e14d36e045058983599
Successfully built pyrouge


In [20]:
!pip install multiprocess

### Данные

Датасет CNN and Daily Mail 

Загрузим предобработанные данные.

In [21]:
%cd ..

/content


In [22]:
# !wget --no-check-certificate --load-cookies /tmp/cookies.txt "http://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'http://docs.google.com/uc?export=download&id=1WE9ZHAW64zKfU41KmUXsLda7zDfT9EqN' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1WE9ZHAW64zKfU41KmUXsLda7zDfT9EqN" -O bertsum_data.zip && rm -rf /tmp/cookies.txt
!wget --no-check-certificate --load-cookies /tmp/cookies.txt "http://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'http://docs.google.com/uc?export=download&id=1x0d61LP9UAN389YN00z0Pv-7jQgirVg6' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1x0d61LP9UAN389YN00z0Pv-7jQgirVg6" -O bertsum_data.zip && rm -rf /tmp/cookies.txt

URL transformed to HTTPS due to an HSTS policy
--2020-11-07 17:07:07--  https://docs.google.com/uc?export=download&confirm=JL3y&id=1x0d61LP9UAN389YN00z0Pv-7jQgirVg6
Resolving docs.google.com (docs.google.com)... 172.217.7.206, 2607:f8b0:4004:806::200e
Connecting to docs.google.com (docs.google.com)|172.217.7.206|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0o-50-docs.googleusercontent.com/docs/securesc/hrliplfd19njvlcnuv5un2pq1k1gbuhq/2c55arvbeeulvhc21vjdj3s5d16hhhre/1604768775000/02403291851892694101/08931067858893519651Z/1x0d61LP9UAN389YN00z0Pv-7jQgirVg6?e=download [following]
--2020-11-07 17:07:07--  https://doc-0o-50-docs.googleusercontent.com/docs/securesc/hrliplfd19njvlcnuv5un2pq1k1gbuhq/2c55arvbeeulvhc21vjdj3s5d16hhhre/1604768775000/02403291851892694101/08931067858893519651Z/1x0d61LP9UAN389YN00z0Pv-7jQgirVg6?e=download
Resolving doc-0o-50-docs.googleusercontent.com (doc-0o-50-docs.googleusercontent.com)... 172.217.15.97, 

In [23]:
!git clone https://github.com/nlpyang/BertSum

Cloning into 'BertSum'...
remote: Enumerating objects: 301, done.
remote: Total 301 (delta 0), reused 0 (delta 0), pack-reused 301
Receiving objects: 100% (301/301), 15.03 MiB | 19.74 MiB/s, done.
Resolving deltas: 100% (174/174), done.


In [24]:
%cd BertSum

/content/BertSum


In [25]:
!unzip ../bertsum_data.zip -d ./bert_data

Archive:  ../bertsum_data.zip
  inflating: ./bert_data/cnndm.test.0.bert.pt  
  inflating: ./bert_data/cnndm.test.1.bert.pt  
  inflating: ./bert_data/cnndm.test.2.bert.pt  
  inflating: ./bert_data/cnndm.test.3.bert.pt  
  inflating: ./bert_data/cnndm.test.4.bert.pt  
  inflating: ./bert_data/cnndm.test.5.bert.pt  
  inflating: ./bert_data/cnndm.train.0.bert.pt  
  inflating: ./bert_data/cnndm.train.100.bert.pt  
  inflating: ./bert_data/cnndm.train.101.bert.pt  
  inflating: ./bert_data/cnndm.train.102.bert.pt  
  inflating: ./bert_data/cnndm.train.103.bert.pt  
  inflating: ./bert_data/cnndm.train.104.bert.pt  
  inflating: ./bert_data/cnndm.train.105.bert.pt  
  inflating: ./bert_data/cnndm.train.106.bert.pt  
  inflating: ./bert_data/cnndm.train.107.bert.pt  
  inflating: ./bert_data/cnndm.train.108.bert.pt  
  inflating: ./bert_data/cnndm.train.109.bert.pt  
  inflating: ./bert_data/cnndm.train.10.bert.pt  
  inflating: ./bert_data/cnndm.train.110.bert.pt  
  inflating: ./bert_da

In [26]:
%cd src

/content/BertSum/src


Пример входных данных

In [27]:
import torch
cnn_test_samp = torch.load("/content/BertSum/bert_data/cnndm.test.0.bert.pt")

In [28]:
cnn_test_samp0 = cnn_test_samp[0]

In [29]:
cnn_test_samp0.keys()

dict_keys(['src', 'labels', 'segs', 'clss', 'src_txt', 'tgt_txt'])

In [30]:
print(cnn_test_samp0['clss']) # индексы CLS токенов для предложений входного текста 
print(cnn_test_samp0['labels']) # таргет метки для предложений (1 - входит в summary, 0 - не входит)
print(cnn_test_samp0['segs']) # id сегментов предложений 
print(cnn_test_samp0['src']) # id слов

[0, 25, 57, 78, 112, 136, 174, 197, 223, 245, 285, 301, 337, 358, 382, 416, 452]
[0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,

In [31]:
cnn_test_samp0['src_txt'] # входной текст


['a university of iowa student has died nearly three months after a fall in rome in a suspected robbery attack in rome .',
 'andrew mogni , 20 , from glen ellyn , illinois , had only just arrived for a semester program in italy when the incident happened in january .',
 'he was flown back to chicago via air ambulance on march 20 , but he died on sunday .',
 'andrew mogni , 20 , from glen ellyn , illinois , a university of iowa student has died nearly three months after a fall in rome in a suspected robbery',
 'he was taken to a medical facility in the chicago area , close to his family home in glen ellyn .',
 "he died on sunday at northwestern memorial hospital - medical examiner 's office spokesman frank shuftan says a cause of death wo n't be released until monday at the earliest .",
 'initial police reports indicated the fall was an accident but authorities are investigating the possibility that mogni was robbed .',
 "on sunday , his cousin abby wrote online : ` this morning my cous

In [32]:
cnn_test_samp0['tgt_txt'] # target summary

'andrew mogni , 20 , from glen ellyn , illinois , had only just arrived for a semester program when the incident happened in january<q>he was flown back to chicago via air on march 20 but he died on sunday<q>initial police reports indicated the fall was an accident but authorities are investigating the possibility that mogni was robbed<q>his cousin claims he was attacked and thrown 40ft from a bridge'

Обучение модели

In [38]:
!python train.py -mode train -encoder classifier -dropout 0.1 -bert_data_path ../bert_data/cnndm -model_path ../models/bert_classifier -lr 2e-3 -visible_gpus 0  -gpu_ranks 0 -world_size 1 -report_every 50 -save_checkpoint_steps 1000 -batch_size 3000 -decay_method noam -train_steps 5000 -accum_count 2 -log_file ../logs/bert_classifier -use_interval true -warmup_steps 10000

[2020-11-07 17:21:38,633 INFO] Device ID 0
[2020-11-07 17:21:38,633 INFO] Device cuda
[2020-11-07 17:21:38,674 INFO] loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ../temp/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
[2020-11-07 17:21:38,675 INFO] extracting archive file ../temp/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpjfjnwr2p
[2020-11-07 17:21:42,603 INFO] Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

[2020-11-07 17:21:49,181 INFO] Summarizer(
  (bert): 

Тестирование на валидационных и тестовых данных

In [35]:
!python train.py -mode validate -bert_data_path ../bert_data/cnndm -model_path ../models/bert_classifier  -visible_gpus 0  -gpu_ranks 0 -batch_size 30000  -log_file ../logs/bert_classifier_valid  -result_path ../results/cnndm -test_all -block_trigram true

[2020-11-07 17:19:31,738 INFO] PPL []


Примеры summary

In [37]:
# extracted summary
N = 20
with open("/content/BertSum/results/cnndm_step5000.candidate") as f:
    dec = [next(f) for x in range(N)]

FileNotFoundError: ignored

In [ ]:
# target summary
N = 20
with open("/content/BertSum/results/cnndm_step5000.gold") as f:
    ref = [next(f) for x in range(N)]

In [ ]:
ref[0].split('<q>')

['the 79th masters tournament gets underway at augusta national on thursday',
 'rory mcilroy and tiger woods will be the star attractions in the field bidding for the green jacket at 2015 masters',
 'mcilroy , justin rose , ian poulter , graeme mcdowell and more gave sportsmail the verdict on each hole at augusta',
 'click on the brilliant interactive graphic below for details on each hole of the masters 2015 course',
 'click here for all the latest news from the masters 2015\n']

In [ ]:
dec[0].split('<q>')

['to help get you in the mood for the first major of the year , rory mcilroy , ian poulter , graeme mcdowell and justin rose , plus past masters champions nick faldo and charl schwartzel , give the lowdown on every hole at the world-famous augusta national golf club .',
 'the masters 2015 is almost here .',
 'click on the graphic below to get a closer look at what the biggest names in the game will face when they tee off on thursday .\n']

In [ ]:
ref[1].split('<q>')

["jeff powell looks ahead to saturday 's fight at the mgm grand",
 'floyd mayweather takes on manny pacquiao in $ 300m showdown',
 'both fighters arrived in las vegas on tuesday with public appearances',
 'read : mayweather makes official arrival ahead of manny pacquiao fight',
 'al haymon : the man behind mayweather who is revolutionising boxing',
 "mayweather vs pacquiao takes centre stage ... but who 's on the undercard ?\n"]

In [ ]:
dec[1].split('<q>')

["sportsmail 's boxing correspondent jeff powell looks ahead to saturday 's mega-fight at the mgm grand after witnessing floyd mayweather and manny pacquiao 's grand arrivals in las vegas .",
 'both boxers made public appearances on tuesday as their $ 300million showdown draws ever closer , and our man powell was there .',
 "powell reflects on the pair 's arrivals on the las vegas strip and looks forward to the rest of the week .\n"]

In [ ]:
ref[2].split('<q>')

['gary locke has been interim manager since start of february',
 'locke has won two and drawn four of his seven games in charge',
 'the 37-year-old took over when allan johnston quit\n']

In [ ]:
dec[2].split('<q>')

['kilmarnock interim manager gary locke has been given the role on a permanent basis after signing a three-year deal .',
 'the former hearts boss joined the club as assistant boss to allan johnston last summer but took control of the team when his ex-tynecastle team-mate quit at the start of february .',
 'the 39-year-old - who will speak at a press conference on friday morning - has lost just once in seven games since taking over at rugby park .\n']